<a href="https://colab.research.google.com/github/brainhj2/Section1Project1/blob/main/N323_lanugage_modeling_with_rnn/N323a_Language_Modeling_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 3 / SPRINT 2 / Assignment 3*

--- 

# Language Modeling with RNN



## Code

다음 링크는 LSTM을 사용하여 Spam 메시지 분류를 수행한 캐글 노트북입니다. => [Link](https://www.kaggle.com/kredy10/simple-lstm-for-text-classification) <br/>

위 노트북에서 사용한 코드를 참고하여<br/>
캐글 데이터셋인 [Women's E-Commerce Clothing Reviews](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews) 를 분류해 보세요.

- 분류에 사용될 텍스트 데이터 : **`Review Text`** 열을 사용합니다.
- 레이블(label) 데이터 : **`Recommended IND`** 열을 사용합니다.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
%matplotlib inline

In [22]:
from google.colab import files

file = files.upload()

Saving Womens Clothing E-Commerce Reviews.csv to Womens Clothing E-Commerce Reviews (1).csv


### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다.
- 필요없는 열(column)을 삭제합니다.

In [61]:
np.random.seed(42)
tf.random.set_seed(42)

In [73]:
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [74]:
df= df[['Review Text', 'Recommended IND']]

In [75]:
df.isnull().sum()

Review Text        845
Recommended IND      0
dtype: int64

In [76]:
df.dropna(subset=['Review Text'], inplace=True)
df.reset_index(drop=True, inplace=True)

<ipython-input-76-b66c36a0a284>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['Review Text'], inplace=True)


In [77]:
df

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
22636,I was very happy to snag this dress at such a ...,1
22637,"It reminds me of maternity clothes. soft, stre...",1
22638,"This fit well, but the top was very see throug...",0
22639,I bought this dress for a wedding i have this ...,1


In [78]:
df.isnull().sum()

Review Text        0
Recommended IND    0
dtype: int64

In [79]:
label= 'Recommended IND'

X= df['Review Text']
y= df[label]

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 20%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words=3000` 으로 설정합니다.
- pad_sequence의 `maxlen=400` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=128, epochs=10, validation_split=0.2` 로 설정합니다.
- EarlyStopping을 적용합니다. 파라미터는 `monitor='val_loss',min_delta=0.0001, patience=3` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [81]:
from __future__ import print_function
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [82]:
####
num_words_3000 = 3000
####

### Tokenizer Instance

tokenizer = Tokenizer(num_words = num_words_3000)
 
tokenizer.fit_on_texts(X_train)

#X_train -> encoded -> padded 

#encoding
X_train_encoded = tokenizer.texts_to_sequences(X_train)

#padding

####
maxlen=400
####

X_train=sequence.pad_sequences(X_train_encoded, maxlen=maxlen, padding='post')

#np.array(y_ target)
y_train=np.array(y_train)

#doing the same thing, for X_test

#encoding
X_test_encoded = tokenizer.texts_to_sequences(X_test)

#padding 
X_test=sequence.pad_sequences(X_test_encoded, maxlen=maxlen, padding='post')

#np.array(y_ target)
y_test=np.array(y_test)


print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)

X_train shape:  (18112, 400)
X_test shape:  (4529, 400)


#RNN 
Define the RNN structure.

In [89]:
max_words= 3000

In [90]:
def RNN():
    inputs = Input(name='inputs',shape=[maxlen])
    layer = Embedding(max_words,50,input_length=maxlen)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [91]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 400)]             0         
                                                                 
 embedding (Embedding)       (None, 400, 50)           150000    
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 FC1 (Dense)                 (None, 256)               33024     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [94]:
# batch_size=128, epochs=10, validation_split=0.2 로 설정합니다.
# EarlyStopping을 적용합니다. 파라미터는 monitor='val_loss',min_delta=0.0001, patience=3 

In [95]:
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001, patience = 3)])

Epoch 1/10
114/114 [==============================] - 11s 30ms/step - loss: 0.4827 - accuracy: 0.8191 - val_loss: 0.5129 - val_accuracy: 0.8073
Epoch 2/10
114/114 [==============================] - 3s 26ms/step - loss: 0.4734 - accuracy: 0.8212 - val_loss: 0.4904 - val_accuracy: 0.8073
Epoch 3/10
114/114 [==============================] - 3s 26ms/step - loss: 0.4724 - accuracy: 0.8212 - val_loss: 0.4933 - val_accuracy: 0.8073
Epoch 4/10
114/114 [==============================] - 3s 25ms/step - loss: 0.4725 - accuracy: 0.8212 - val_loss: 0.4901 - val_accuracy: 0.8073
Epoch 5/10
114/114 [==============================] - 3s 25ms/step - loss: 0.4714 - accuracy: 0.8212 - val_loss: 0.4945 - val_accuracy: 0.8073
Epoch 6/10
114/114 [==============================] - 3s 26ms/step - loss: 0.4713 - accuracy: 0.8212 - val_loss: 0.4938 - val_accuracy: 0.8073
Epoch 7/10
114/114 [==============================] - 3s 26ms/step - loss: 0.4713 - accuracy: 0.8212 - val_loss: 0.4923 - val_accuracy: 0.807

In [99]:
accuracy = model.evaluate(X_test,y_test)

142/142 [==============================] - 1s 8ms/step - loss: 0.4753 - accuracy: 0.8207


In [100]:
print(f'Test accuracy: {accuracy}')

Test accuracy: [0.47531554102897644, 0.8207109570503235]


In [92]:
# import tensorflow as tf

# # model을 정의합니다.
# """
# keras의 기본 Embedding 벡터를 사용하였으며
# LSTM 층에 dropout/recurrent_dropout을 적용하였습니다.
# """
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Embedding(max_features, 128), # Embedding Layer를 거친 후의 shape : (batch_size, maxlen, embedding_size=128)
#   tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2), # LSTM Layer를 거친 후의 shape : (batch_size, 1, hidden_size=128)
#   tf.keras.layers.Dense(1, activation='sigmoid')  # 긍정/부정 감정을 예측하는 것이기 때문에 1개의 노드 및 sigmoid를 사용합니다.
# ])

# model.compile(loss='binary_crossentropy',
#               optimizer='adam', 
#               metrics=['accuracy'])

# model.summary()